<a href="https://colab.research.google.com/github/lohan20s/Data-Analysis-with-Python-mean_var/blob/main/fcc_sms_text_classification_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.0/658.0 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-04-04 21:04:00--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.005s  

2025-04-04 21:04:00 (66.9 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-04-04 21:04:00--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [3]:
#load train and test data
train_dataset=pd.read_csv(train_file_path,sep="\t",header=None)
train_labels=train_dataset.pop(train_dataset.columns[0])
train_labels_int= train_labels.map({'ham': 1, 'spam': 0})

test_dataset=pd.read_csv(test_file_path,sep="\t",header=None)
test_labels=test_dataset.pop(test_dataset.columns[0])
test_labels_int= test_labels.map({'ham': 1, 'spam': 0})



In [4]:
#encode words as integers by creating a dictionary from the full dataset
tokenizer = keras.preprocessing.text.Tokenizer(filters='') #ensure filters are off, so that phone numbers are not filtered out
combineddata=pd.concat([train_dataset,test_dataset])

tokenizer.fit_on_texts(combineddata.squeeze())
word_index = tokenizer.word_index  # Dictionary of words -> integer index
VOCAB_SIZE = len(word_index)
train_data = tokenizer.texts_to_sequences(train_dataset.squeeze())  # Convert sentences to sequences of integers
test_data = tokenizer.texts_to_sequences(test_dataset.squeeze())  # Convert sentences to sequences of integers


In [5]:
#pad sequences so that all are the same length
max_train_length = max(len(seq) for seq in train_data)
max_test_length = max(len(seq) for seq in test_data)
MAXLEN=max(max_train_length,max_test_length)

Pad_sequence=keras.preprocessing.sequence.pad_sequences
train_data = Pad_sequence(train_data, MAXLEN)
test_data = Pad_sequence(test_data, MAXLEN)

In [6]:
#create a model with LSTM for sequential data
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE+1, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
#compile and fit the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
model.fit(train_data, train_labels_int, epochs=10,validation_data=(test_data, test_labels_int))
model.summary()

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 24s 143ms/step - acc: 0.8892 - loss: 0.3115 - val_acc: 0.9698 - val_loss: 0.0952
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 17s 118ms/step - acc: 0.9726 - loss: 0.0853 - val_acc: 0.9792 - val_loss: 0.0709
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 21s 120ms/step - acc: 0.9874 - loss: 0.0499 - val_acc: 0.9820 - val_loss: 0.0602
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 118ms/step - acc: 0.9923 - loss: 0.0326 - val_acc: 0.9828 - val_loss: 0.0590
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - acc: 0.9937 - loss: 0.0257 - val_acc: 0.9835 - val_loss: 0.0657
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 130ms/step - acc: 0.9953 - loss: 0.0214 - val_acc: 0.9777 - val_loss: 0.0773
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 16s 120ms/step - acc: 0.9971 - loss: 0.0120 - val_acc: 0.9777 - val_loss: 0.0842
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 117ms/step - acc: 0.9976 - loss: 0.0124 - val_acc: 0.9806 - val_loss: 0.0772
Epoch 9/10
131/131 ━━━━━━━━━━━━━

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 171, 64)             │         863,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,792,260 (6.84 MB)

 Trainable params: 896,129 (3.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 896,131 (3.42 MB)

In [8]:
#print accuracy of the model on the test dataset
results = model.evaluate(test_data, test_labels_int)
print(results)

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - acc: 0.9829 - loss: 0.0936
[0.08548766374588013, 0.9841954112052917]


In [34]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  str=[pred_text]
  str_int=tokenizer.texts_to_sequences(str)
  str_int = keras.preprocessing.sequence.pad_sequences(str_int, MAXLEN)
  results=model.predict(str_int)
  prediction=[]
  prediction.append(results[0][0])
  if results[0][0]>0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)

pred_text = "how are you doing today"
prediction = predict_message(pred_text)
print("The text message " + "is " + str(prediction[0]*100) +"% " + str(prediction[1]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
The text message is 99.99001% ham


In [20]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
You passed the challenge. Great job!
